# Working with a database

This week's assignment has a few basic steps.  First, we're going to pull some data down off the internet and store it into our MySQL database.  Make sure that you use your username as part of the table name as show in the examples so that you don't create a problem for other students.

Then, we'll merge that with some data already in the database and calculate a few results.  When it comes to calculating the results, you can do so either with SQL or with Pandas operations.


## PART 1: Setup your database connection and table name

In the code below, change the value of the variable `MYTABLE` to use your own username rather that `'pboal'`

You can then use `MYTABLE` in the rest of your code to reference that table name.

In [1]:
#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!
# First, put in your user name below
#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!

MYTABLE='ashley' + '_data'


# Then, create a connection to the same database, using the 
# same user name and password in the `mysql-examples.ipynb` 
# file.  Use conn as the database connection variable.
import os
import pymysql
from sqlalchemy import create_engine
import pandas as pd

host = 'slucor2020.cgdcoitnku0k.us-east-1.rds.amazonaws.com'
port = '3306'
user = 'slucor2020'
password = 'SLUcor2020'
database = 'hds5210'

conn = create_engine('mysql+pymysql://' + 
                     user + ':' + 
                     password + '@' + 
                     host + '/' + 
                     database, echo=False)

In [2]:
assert(MYTABLE != 'paulboal_data')
assert(conn.connect())

## PART 2: Bring in outside data

Grab the data from this URL and put it into a database table named with your `username_data`.

http://dhcs-chhsagency.opendata.arcgis.com/datasets/8e4f3a0c75b9424d888d11c1f949cc32_0.csv

By the end of your cell, the table should be created.  The tests are going to verify that the table exists and looks right.

In [9]:
df.columns

Index(['Provider_Name', 'NPI', 'CCN', 'Medicaid_EP_Hospital_Type',
       'Street_Address', 'City', 'County', 'State', 'Zip_Code',
       'Payment_Year_Number', 'Program_Type', 'Total_payments',
       'Last_Program_Year', 'Last_Payment_Year', 'Last_Payment_Criteria',
       'Most_Recent_Disbursement_Amount', 'LONGITUDE', 'LATITUDE', 'FID'],
      dtype='object')

In [12]:
url = 'http://dhcs-chhsagency.opendata.arcgis.com/datasets/8e4f3a0c75b9424d888d11c1f949cc32_0.csv'
df = pd.read_csv(url)

In [17]:
#I dont know why.
df.insert(0,'Y', None)
df.insert(0,'X', None)

In [20]:
df.to_sql(MYTABLE, conn)

In [21]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
assert(dxyz.shape == (328,22))
assert(list(dxyz.columns) == ['index', 'X', 'Y', 'Provider_Name', 'NPI', 'CCN',
       'Medicaid_EP_Hospital_Type', 'Street_Address', 'City', 'County',
       'State', 'Zip_Code', 'Payment_Year_Number', 'Program_Type',
       'Total_payments', 'Last_Program_Year', 'Last_Payment_Year',
       'Last_Payment_Criteria', 'Most_Recent_Disbursement_Amount', 'LONGITUDE',
       'LATITUDE', 'FID'])

## PART 3: Combine with other data in the database

In the database, there is an existing table called `population`.  We want to merge the DHCS datafile loaded above with the population data available in this other database table  The tables can be merged on `MYTABLE`'s `Zip_Code` field and `population`'s `zipcode` field.

Note that not all `Zip_Codes` from your downloaded file have to be in the `population` table.  If they aren't, then I want you to eliminate the non-matching records.  That is, only keep the records that have a matching ZIP code in both sets of data.

Answer the question:
Which providers are located in the zipcode with the largest population?

Put your answer in the form `answer = ['a', 'list', 'of', 'NPI', 'like', '1593042103]`

In [22]:
population = pd.read_sql_query('SELECT * FROM ' + 'population', conn)

df.Zip_Code = df.Zip_Code.astype(str)

df.NPI = df.NPI.astype(str)

merge_df = df.merge(population, right_on='zipcode', left_on='Zip_Code', how='inner')

merge_df.loc[merge_df.population.idxmax(),'NPI']

### SOLUTION GOES HERE
answer = [merge_df.loc[merge_df.population.idxmax(),'NPI']]

In [55]:
assert(type(answer) == list)
assert(answer == ['1194016923'])

## PART 4: Total by hospital type

This is a multistep process:
* From our downloaded data file, compute the `Total payments` per ZIP code and Medicaid EP Hospital Type.
* Then merge that with the `population` data to compute a `Total payments` per person.
* Then average that across all of the `Medicaid EP Hospital Types` to get an average per persona payment for these type of hospital.

Your answer should be in structure of a data frame with at least two columns:
* Medicaid_EP_Hospital_Type
* Avg_Pay_per_Capita

In [101]:
total_payments = df.groupby(['Zip_Code','Medicaid_EP_Hospital_Type'])['Total_payments'].sum().reset_index()

In [102]:
merge_total = total_payments.merge(population, right_on='zipcode', left_on='Zip_Code', how='inner')

In [103]:
merge_total.head()

,Zip_Code,Medicaid_EP_Hospital_Type,Total_payments,zipcode,population,median_age,total_males,total_females,total_households,avg_household
0,90004,Acute Care Hospitals,1292117,90004,62180,34.8,31302,30878,22547,2.73
1,90012,Acute Care Hospitals,3840496,90012,31103,36.3,19493,11610,10327,2.12
2,90015,Acute Care Hospitals,5580824,90015,18986,31.3,9833,9153,7420,2.45
3,90017,Acute Care Hospitals,4195996,90017,23768,29.4,12818,10950,9338,2.53
4,90018,Acute Care Hospitals,1553491,90018,49310,33.2,23770,25540,15493,3.12


In [104]:
merge_total['Avg_Pay_per_Capita'] = merge_total['Total_payments']/merge_total['population']

In [105]:
merge_total = merge_total.groupby(['Medicaid_EP_Hospital_Type']).Avg_Pay_per_Capita.mean().reset_index()

In [106]:
merge_total

,Medicaid_EP_Hospital_Type,Avg_Pay_per_Capita
0,Acute Care Hospitals,17974.362790
1,Children's Hospitals,111.559021


In [107]:
### SOLUTION GOES HERE
answer = merge_total

In [108]:
assert(type(answer) == pd.core.frame.DataFrame)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Acute Care Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 17974.363)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Children\\'s Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 111.559)
